In [3]:
import re
import pickle 
import sqlite3 as sl
from unicodedata import normalize
import pandas as pd
from pathlib import Path
from sklearn.naive_bayes import BernoulliNB
from scipy import sparse
import numpy as np
con = sl.connect('../coleta_de_dados/banco_scraping_olx.db')

In [4]:
## Carregando o modelo

In [5]:
with open('./BernoulliNB_model.pkl', 'rb') as file:
    clf = pickle.load(file)

with open('./BernoulliNB_model_adds.pkl', 'rb') as file:
    model_add_data = pickle.load(file)   

lista_todas_palavras = model_add_data['lista_todas_palavras'] 
lista_precos = model_add_data['lista_precos'] 
lista_todas_categorias = model_add_data['lista_todas_categorias']    

In [6]:
## Usando o Modelo

In [7]:
def normaliza_texto_vetorizado(text:str)->str: 
    """
    função criada para transformar os títulos dos anuncios em um formato padronizado, sem caracteres especiais e sem números
    """
    frase_minuscula = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower() #normaliza para retirar acentos e transforma para minuscula
    somente_palavras_validas = re.sub("([^\w]+)|((\w|d)*[\d](\w|d)*)"," ",frase_minuscula) #substitui a string acima por uma string sem numeros e sem caracteres especiais
    texto_normalizado = re.sub("\s+"," ",somente_palavras_validas).strip() #substitui um ou mais espaços da string acima por um único espaço
    return texto_normalizado.split(" ") #retorna uma lista com as palavras válidas 

def categoria_para_indice(categoria,lista_todas_categorias):
    """verifica qual o índice da categoria informada conforme a lista_todas_categorias"""
    if lista_todas_categorias.count(categoria): #se existir a categoria na lista,
        return lista_todas_categorias.index(categoria) #retorna o indice da categoria na lista_todas_categorias
    else:
        return -1 #caso contrário, retorna -1
        
def titulo_para_indices(titulo,lista_todas_palavras): 
    """verifica qual o indice """
    lista_palavras_titulo = normaliza_texto_vetorizado(titulo) #normaliza o título informado
    indices_titulo =[] #lista de indices (conforme lista_todas_palavras) referentes as palavras do título
    for palavra_titulo in lista_palavras_titulo: #para cada palavra na lista_palavras_titulo,
        if lista_todas_palavras.count(palavra_titulo): #se existir a palavra do título na lista lista_todas_palavras, 
            indices_titulo.append(lista_todas_palavras.index(palavra_titulo)) #adiciona o indice da palavra(indice referente a lista_todas_palavras) na indices_titulo
    return indices_titulo   #retorna uma lista com os indices do titulo

def preco_para_indice(preco,indice_start=0):
    """
    Atribui um índice para um range de preços na escala log
    """
    if isinstance(preco,float) or isinstance(preco,int): #se o preço for int ou float retorna true e segue
        if preco>1:
            index = np.ceil(np.log10(preco)) #arredonda o valor de Log(preço) na base 10 para servir como um índice
            index = int(index) if index>=0 and index<len(lista_precos) else 0 #passa o indice para inteiro se indice for >= 0 e for menor do que o tamaho da lista_preços
            return index+indice_start #indice_start -> começa depois do termino da matriz de palavras
    return 0              

In [8]:
def procura_categoria(titulo,preco=None,qt_impr=5):
    """ 
    Função para testar as entradas com base no título e no preço do produto a ser cadastrado
    """
    qt_colunas = len(lista_todas_palavras)+len(lista_precos) #qt de colunas da matriz
    XN = sparse.dok_matrix((1, qt_colunas), dtype=np.int8) #constrói matriz esparsa XN de forma incremental
    if preco:
        XN[0,preco_para_indice(preco,len(lista_todas_palavras))] = 1 #popula 1 na coluna referente ao preço

    indices_palavras_titulo = titulo_para_indices(titulo,lista_todas_palavras) #lista com os indices das palavras do titulo
    for indice_palavra in indices_palavras_titulo: 
        XN[0,indice_palavra] = 1 #popula 1 nas colunas referentes às palavras do título com base nos índices de indices_palavras_titulo

    categorias_prob = clf.predict_proba(XN[0]) # retorna a probabilidade referente a cada categoria
    indice_categorias_ordenado = np.flip(np.argsort(categorias_prob)) #ordena os indices de acordo com a probabilidade decrescente
    limite_impressoes = qt_impr #variável que define a quantidade de categorias a serem impressas
    for indice_categoria in indice_categorias_ordenado.tolist()[0]:
        print(f'{lista_todas_categorias[indice_categoria]}:{categorias_prob[0][indice_categoria]*100:.2f}%') #imprime uma categoria e o percentual referente a ela (ordem decrescente)
        if limite_impressoes<=1:
            break
        limite_impressoes -=1

In [9]:
procura_categoria('Chevrolet Onix')

Carros, vans e utilitários:93.02%
Peças para carros, vans e utilitários:6.87%
Caminhões:0.02%
Produção Rural:0.02%
Hobbies e coleções:0.02%
